# Hackathon: From Raw Data to ML-Ready Dataset
## Insight-Driven EDA and End-to-End Feature Engineering on Airbnb Data Using pandas and Plotly

### What is a Hackathon?

A hackathon is a fast-paced, collaborative event where participants use data and technology to solve a real problem end-to-end.  
In this hackathon, you will work with a **real-world Airbnb dataset** and complete two interconnected goals:

- Produce a **high-quality exploratory data analysis (EDA)** using `pandas` and `plotly`, extracting meaningful insights, trends, and signals from the data.  
- Design and deliver a **clean, feature-rich, ML-ready dataset** that will serve as the foundation for a follow-up hackathon focused on building and evaluating machine learning models.

Your task is to **get the most out of the data**: uncover structure and patterns through EDA, and engineer informative features (numerical, categorical, temporal, textual (TF–IDF), and optionally image-based) to maximize the predictive power of the final dataset.

<div class="alert alert-success">
<b>About the Dataset</b>

<u>Context</u>

The data comes from <a href="https://insideairbnb.com/get-the-data/">Inside Airbnb</a>, an open project that publishes detailed, regularly updated datasets for cities around the world.  
Each city provides three main CSV files:

- <b>listings.csv</b> — property characteristics, host profiles, descriptions, amenities, etc.  
- <b>calendar.csv</b> — daily availability and pricing information for each listing.  
- <b>reviews.csv</b> — guest feedback and textual reviews.

These datasets offer a rich view of the short-term rental market, including availability patterns, pricing behavior, host attributes, and guest sentiment.  

<u>Inspiration</u>

Your ultimate objective is to create a dataset suitable for training a machine learning model that predicts whether a specific Airbnb listing will be <b>available on a given date</b>, using property attributes, review information, and host characteristics.
</div>

<div class="alert alert-info">
<b>Task</b>

Using one city of your choice from Inside Airbnb, create an end-to-end pipeline that:

1. Loads and explores the raw data (EDA).  
2. Engineers features (numerical, categorical, temporal, textual TF–IDF, etc.).  
3. Builds a unified ML-ready dataset.  

Please remember to add comments explaining your decisions. Comments help us understand your thought process and ensure accurate evaluation of your work. This assignment requires code-based solutions—**manually calculated or hard-coded results will not be accepted**. Thoughtful comments and visualizations are encouraged and will be highly valued.

- Write your solution directly in this notebook, modifying it as needed.
- Once completed, submit the notebook in **.ipynb** format via Moodle.
    
<b>Collaboration Requirement: Git & GitHub</b>

You must collaborate with your team using a **shared GitHub repository**.  
Your use of Git is part of the evaluation. We will specifically look at:

- Commit quality (clear messages, meaningful steps).  
- Balanced participation across team members.  
- Use of branches.  
- Ability to resolve merge conflicts appropriately.  
- A clean, readable project history that reflects real collaboration.

Good Git practice is **part of your grade**, not optional.
</div>
<div class="alert alert-danger">
    You are free to add as many cells as you wish as long as you leave untouched the first one.
</div>

<div class="alert alert-warning">

<b>Hints</b>

- Text columns often carry substantial predictive power, use text-vectorization methods to extract meaningful features.  
- Make sure all columns use appropriate data types (categorical, numeric, datetime, boolean). Correct dtypes help prevent subtle bugs and improve performance.  
- Feel free to enrich the dataset with any additional information you consider useful: engineered features, external data, derived temporal features, etc.  
- If the dataset is too large for your computer, use <code>.sample()</code> to work with a subset while preserving the logic of your pipeline.  
- Plotly offers a wide variety of powerful visualizations, experiment creatively, but always begin with a clear analytical question: *What insight am I trying to uncover with this plot?*

</div>




<div class="alert alert-danger">
<b>Submission Deadline:</b> Wednesday, December 3rd, 12:00

Start with a simple, working pipeline.  
Do not over-complicate your code too much. Start with a simple working solution and refine it if you have time.
</div>

<div class="alert alert-danger">
    
You may add as many cells as you want, but the **first cell must remain exactly as provided**. Do not edit, move, or delete it under any circumstances.
</div>


In [1]:
# LEAVE BLANK

### Team Information

Fill in the information below.  
All fields are **mandatory**.

- **GitHub Repository URL**: Paste the link to the team repo you will use for collaboration.
- **Team Members**: List all student names (and emails or IDs if required).

Do not modify the section title.  
Do not remove this cell.


In [2]:
# === Team Information (Mandatory) ===
# Fill in the fields below.

GITHUB_REPO = "https://github.com/Jan2134/PythonHackathon"       # e.g. "https://github.com/myteam/airbnb-hackathon"
TEAM_MEMBERS = [
    "Andres Ramirez",
    "Yiben Fruncillo",
    "Mara Rüsen",
    "Ella Magdic",
    "Jan Erik Sternberg"
]

GITHUB_REPO, TEAM_MEMBERS


('https://github.com/Jan2134/PythonHackathon',
 ['Andres Ramirez',
  'Yiben Fruncillo',
  'Mara Rüsen',
  'Ella Magdic',
  'Jan Erik Sternberg'])

In [3]:
# Base configuration and utility imports
from __future__ import annotations
import json
import math
import textwrap
from pathlib import Path
from typing import List, Optional
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
DATA_DIR = Path.cwd()
DATA_DIR

WindowsPath('c:/Users/frunc/Desktop/ttt')

### Project Roadmap
1. **Data ingestion**: load Amsterdam calendar, listings, and reviews files into memory with appropriate dtypes.
2. **Exploratory analysis**: inspect structure, missingness, and key business metrics, supported by Plotly visuals.
3. **Feature engineering**: clean numeric fields, derive temporal flags, encode categoricals, and extract TF–IDF signals.
4. **Dataset assembly**: merge engineered features into a single ML-ready table targeting listing availability per day.
5. **Export & documentation**: persist the dataset and capture metadata for downstream modeling.

In [4]:
# Load raw Inside Airbnb assets
calendar_path = DATA_DIR / "calendar.csv.gz"
listings_path = DATA_DIR / "listings.csv.gz"
reviews_path = DATA_DIR / "reviews.csv.gz"
raw_calendar = pd.read_csv(calendar_path, parse_dates=["date"], low_memory=False)
raw_listings = pd.read_csv(listings_path, low_memory=False)
raw_reviews = pd.read_csv(reviews_path, parse_dates=["date"], low_memory=False)
raw_calendar.shape, raw_listings.shape, raw_reviews.shape

((3825200, 7), (10480, 79), (501084, 6))

In [5]:
# Inspect representative slices to understand schema and data quality
calendar_preview = raw_calendar.head().drop(columns=["available"]).assign(available_raw=raw_calendar["available"].head())
listings_preview = raw_listings[["id","name","property_type","room_type","accommodates","price"]].head()
reviews_preview = raw_reviews[["listing_id","date","comments"]].head()
calendar_preview, listings_preview, reviews_preview

(   listing_id       date  price  adjusted_price  minimum_nights  \
 0      538723 2025-09-11    NaN             NaN               5   
 1      538723 2025-09-12    NaN             NaN               5   
 2      538723 2025-09-13    NaN             NaN               5   
 3      538723 2025-09-14    NaN             NaN               5   
 4      538723 2025-09-15    NaN             NaN               5   
 
    maximum_nights available_raw  
 0              30             f  
 1              30             f  
 2              30             f  
 3              30             f  
 4              30             f  ,
       id                                               name  \
 0  27886  Romantic, stylish B&B houseboat in canal district   
 1  28871                            Comfortable double room   
 2  29051                   Comfortable single / double room   
 3  44391    Quiet 2-bedroom Amsterdam city centre apartment   
 4  48373                Cozy family home in Amsterdam Sout

In [6]:
# Helper to calculate quick dataset diagnostics
def dataset_overview(df: pd.DataFrame, name: str) -> pd.Series:
    return pd.Series({
        "rows": len(df),
        "columns": df.shape[1],
        "missing_pct": df.isna().mean().mean() * 100,
        "approx_mem_mb": df.memory_usage(deep=True).sum() / 1024**2
    }, name=name)
overview = pd.concat([
    dataset_overview(raw_calendar, "calendar"),
    dataset_overview(raw_listings, "listings"),
    dataset_overview(raw_reviews, "reviews"),
], axis=1)
overview

,calendar,listings,reviews
rows,3.825200e+06,10480.000000,501084.000000
columns,7.000000e+00,79.000000,6.000000
missing_pct,2.857143e+01,10.392912,0.001064
approx_mem_mb,3.575036e+02,41.779018,226.815126


In [7]:
# Core cleaning for calendar signals
calendar = raw_calendar.copy()
calendar['available_flag'] = (calendar['available'] == 't').astype(int)
for price_col in ['price', 'adjusted_price']:
    if price_col in calendar.columns:
        calendar[price_col] = (
            calendar[price_col]
            .astype(str)
            .str.replace('$', '', regex=False)
            .str.replace(',', '', regex=False)
            .replace('nan', np.nan)
            .astype(float)
        )
calendar['listing_id'] = calendar['listing_id'].astype(int)
calendar['nights_available_pct'] = calendar['available_flag'].groupby(calendar['listing_id']).transform('mean')
calendar.head(3)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,available_flag,nights_available_pct
0,538723,2025-09-11,f,NaN,NaN,5,30,0,0.0
1,538723,2025-09-12,f,NaN,NaN,5,30,0,0.0
2,538723,2025-09-13,f,NaN,NaN,5,30,0,0.0


In [8]:
# Exploratory statistics focused on booking availability dynamics
calendar_stats = pd.DataFrame({
    "date_min": [calendar['date'].min()],
    "date_max": [calendar['date'].max()],
    "listings_covered": [calendar['listing_id'].nunique()],
    "median_price": [calendar['price'].median()],
    "availability_rate": [calendar['available_flag'].mean()],
    "median_min_nights": [calendar['minimum_nights'].median()],
})
calendar_stats

,date_min,date_max,listings_covered,median_price,availability_rate,median_min_nights
0,2025-09-11,2026-09-10,10480,NaN,0.257678,3.0


In [9]:
# Share of days with price information after cleaning
price_quality = calendar['price'].notna().mean()
adjusted_price_quality = calendar['adjusted_price'].notna().mean() if 'adjusted_price' in calendar else np.nan
price_quality, adjusted_price_quality

(np.float64(0.0), np.float64(0.0))

In [10]:
raw_calendar['price'].dropna().head()

Series([], Name: price, dtype: float64)

In [11]:
raw_calendar.columns.tolist()

['listing_id',
 'date',
 'available',
 'price',
 'adjusted_price',
 'minimum_nights',
 'maximum_nights']

In [12]:
raw_listings.columns.tolist()

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'source',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30

#### Listing feature engineering overview
Convert host metrics, pricing, bathrooms, and review timestamps into numeric features ready for modeling.

In [13]:
# Derive engineered listing-level features
def to_float(series: pd.Series) -> pd.Series:
    # Gracefully coerce mixed-type inputs (str, float, missing) into numeric values
    return pd.to_numeric(series, errors='coerce')
def parse_currency(series: pd.Series) -> pd.Series:
    # Remove currency symbols/thousands separators before converting to float
    return (
        series.astype(str)
        .str.replace('$', '', regex=False)
        .str.replace(',', '', regex=False)
        .replace({'nan': np.nan})
        .pipe(to_float)
    )
def parse_percent(series: pd.Series) -> pd.Series:
    # Strip percent signs and scale into 0–1 range
    return (
        series.astype(str)
        .str.replace('%', '', regex=False)
        .replace({'nan': np.nan, 'None': np.nan})
        .pipe(to_float)
    ) / 100.0
def extract_bathrooms(series: pd.Series) -> pd.Series:
    # Capture the numeric portion of strings like "1.5 baths"
    return (
        series.astype(str)
        .str.extract(r'([0-9]+\.?[0-9]*)', expand=False)
        .pipe(to_float)
    )
# Select fields that contain host behaviour, property size, and review history
listings = raw_listings[[
    'id','host_is_superhost','host_response_time','host_response_rate','host_acceptance_rate',
    'host_listings_count','host_total_listings_count','property_type','room_type','accommodates',
    'bathrooms_text','bedrooms','beds','amenities','price','minimum_nights','maximum_nights',
    'number_of_reviews','number_of_reviews_l30d','number_of_reviews_ltm','reviews_per_month',
    'review_scores_rating','review_scores_cleanliness','review_scores_communication',
    'availability_30','availability_60','availability_90','availability_365','instant_bookable',
    'neighbourhood_cleansed','neighbourhood_group_cleansed','latitude','longitude',
    'first_review','last_review'
]].copy()
listings.rename(columns={'id': 'listing_id'}, inplace=True)
# Convert key categorical flags into 0/1 indicators for downstream models
listings['host_is_superhost'] = listings['host_is_superhost'].eq('t').astype(int)
listings['instant_bookable'] = listings['instant_bookable'].eq('t').astype(int)
# Normalize host performance metrics
listings['host_response_rate'] = parse_percent(listings['host_response_rate'])
listings['host_acceptance_rate'] = parse_percent(listings['host_acceptance_rate'])
# Currency fields are sourced from listings.csv because calendar prices are blank
listings['price'] = parse_currency(listings['price'])
# Bathroom counts appear as text in the raw dump (e.g. "Half-bath")
listings['bathrooms'] = extract_bathrooms(listings.pop('bathrooms_text'))
# Amenity strings are JSON-like; count unique tokens per listing
listings['amenities_count'] = listings['amenities'].fillna('[]').str.strip('{}[]').str.count(',') + 1
listings['amenities_count'] = listings['amenities_count'].mask(listings['amenities'].isna(), np.nan)
# Convert review timestamps for freshness features
listings['first_review'] = pd.to_datetime(listings['first_review'])
listings['last_review'] = pd.to_datetime(listings['last_review'])
listings['days_since_last_review'] = (calendar['date'].max() - listings['last_review']).dt.days
# Numeric review quality scores feed into baseline models
listings['review_scores_rating'] = to_float(listings['review_scores_rating'])
listings['review_scores_cleanliness'] = to_float(listings['review_scores_cleanliness'])
listings['review_scores_communication'] = to_float(listings['review_scores_communication'])
# Preserve nightly price under a modeling-friendly name
listings['target_price'] = listings['price']
# Quick peek confirms transformations landed as expected
listings_features_preview = listings.head(3)
listings_features_preview

,listing_id,host_is_superhost,host_response_time,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,property_type,room_type,accommodates,...,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,first_review,last_review,bathrooms,amenities_count,days_since_last_review,target_price
0,27886,1,within an hour,1.0,0.97,1.0,1.0,Private room in houseboat,Private room,2,...,Centrum-West,NaN,52.38761,4.89188,2012-01-09,2025-09-07,1.5,51,368.0,132.0
1,28871,1,within an hour,1.0,0.99,2.0,2.0,Private room in rental unit,Private room,2,...,Centrum-West,NaN,52.36775,4.89092,2010-08-22,2025-09-07,1.0,18,368.0,89.0
2,29051,1,within an hour,1.0,0.99,2.0,2.0,Private room in condo,Private room,2,...,Centrum-Oost,NaN,52.36584,4.89111,2011-03-16,2025-09-08,1.0,18,367.0,61.0


### Feautures starts
Fine-tunes amenity counts and densities so listings with empty amenity lists do not inflate feature values.

In [14]:
# Refine amenity counts to avoid inflated values for empty braces
def count_amenities(raw: str) -> int:
    # Count only non-empty amenity tokens, handling blanks and NaNs safely
    if not isinstance(raw, str):
        return 0
    stripped = raw.strip('{}[]')
    if not stripped:
        return 0
    return sum(1 for amenity in stripped.split(',') if amenity.strip())
# Amenities per guest is a useful proxy for host investment in the listing
listings['amenities_count'] = listings['amenities'].apply(count_amenities)
listings['amenities_density'] = listings['amenities_count'] / listings['accommodates'].replace(0, np.nan)
listings[['listing_id', 'amenities', 'amenities_count', 'amenities_density']].head(3)

,listing_id,amenities,amenities_count,amenities_density
0,27886,"[""Canal view"", ""Coffee"", ""Private backyard \u2...",51,25.5
1,28871,"[""Heating"", ""Lock on bedroom door"", ""Hair drye...",18,9.0
2,29051,"[""Heating"", ""Lock on bedroom door"", ""Hair drye...",18,9.0


In [15]:
# Interactive visuals to surface pricing and availability signals
calendar['month'] = calendar['date'].dt.to_period('M').astype(str)
monthly_availability = (
    calendar.groupby('month')['available_flag']
    .mean()
    .reset_index(name='availability_rate')
)
monthly_figure = px.line(
    monthly_availability,
    x='month',
    y='availability_rate',
    title='Availability Rate by Month',
    markers=True,
)
monthly_figure.update_layout(xaxis_title='Month', yaxis_title='Share of Available Nights')
monthly_figure

In [16]:
# Focus the price distribution on the central mass and call out trimmed tails
price_series = listings['price'].dropna()
if price_series.empty:
    raise ValueError('Listing price column is empty after cleaning.')
p01, p99 = np.percentile(price_series, [1, 99])
price_focus = price_series[price_series.between(p01, p99)]
tail_share_pct = (1 - len(price_focus) / len(price_series)) * 100
price_focus_fig = px.histogram(
    price_focus.to_frame(name='price'),
    x='price',
    nbins=50,
    title='Listing Nightly Rate Distribution (1st-99th Percentile)',
    marginal='box',
    histnorm='probability density',
    opacity=0.85,
    color_discrete_sequence=['#2A9D8F']
 )
price_focus_fig.add_vline(
    x=price_series.median(),
    line_dash='dash',
    line_color='#E76F51',
    annotation_text='Median',
    annotation_position='top right'
)
price_focus_fig.add_annotation(
    x=p99,
    y=0.95,
    xref='x',
    yref='paper',
    text=f"Top 1% trimmed ({tail_share_pct:.1f}% of listings)",
    showarrow=False,
    font=dict(size=12),
    xanchor='right'
 )
price_focus_fig.update_layout(
    xaxis_title='Nightly price (USD)',
    yaxis_title='Density of listings',
    bargap=0.05,
    hovermode='x unified'
)
price_focus_fig

#### Availability patterns by day of week
Weekday vs weekend comparisons help identify demand spikes that the ML target should reflect.

In [17]:
# Visualise average availability by day of week to expose temporal demand shifts
dow_summary = (
    calendar.assign(day_of_week=calendar['date'].dt.day_name())
    .groupby('day_of_week')['available_flag']
    .mean()
    .reindex(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
    .reset_index(name='availability_rate')
)
dow_fig = px.bar(
    dow_summary,
    x='day_of_week',
    y='availability_rate',
    title='Average Availability Rate by Day of Week',
    text='availability_rate'
)
dow_fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
dow_fig.update_layout(yaxis_title='Share of available nights', xaxis_title='Day of week', uniformtext_minsize=10, uniformtext_mode='hide', bargap=0.3)
dow_fig

### Price Distribution of Amsterdam Listings
To see where most hosts set their nightly rates we plot the cleaned price distribution for Amsterdam listings.

**Why this matters**
- Shows whether the market is budget, mid-range, or premium heavy
- Flags the ultra-expensive tail that can skew averages
- Informs future transformations such as log-price or price-band features

**Data cleaning steps performed**
- Converted `price` from string to numeric and stripped `$` and commas
- Dropped listings missing a nightly rate
- Trimmed the top and bottom 1% of prices to prevent extreme outliers from dominating the view
- Calculated the share of listings trimmed so we know how much signal was removed

**What the histogram shows**
The histogram concentrates on the central 98% of listings, adds a dashed median reference, and annotates the trimmed tail. This makes it easy to read the typical nightly price range while still acknowledging the small fraction of ultra-luxury listings.

#### Price versus quality signals
Cross-checking price against accommodates and ratings highlights potential undervalued listings.

In [18]:
# Explore pricing relative to capacity and reputation to spot hidden-gem candidates
scatter_source = listings.dropna(subset=['target_price', 'accommodates', 'review_scores_rating'])
sample_count = min(len(scatter_source), 2000)
scatter_sample = scatter_source.sample(sample_count, random_state=42) if sample_count else scatter_source
host_type_labels = scatter_sample['host_is_superhost'].map({1: 'Superhost', 0: 'Regular host'})
price_scatter = px.scatter(
    scatter_sample,
    x='accommodates',
    y='target_price',
    color=host_type_labels,
    size='review_scores_rating',
    hover_data={'listing_id': True, 'review_scores_rating': True, 'amenities_count': True},
    title='Nightly Price vs. Capacity (Bubble size = Review Rating)',
)
price_scatter.update_layout(xaxis_title='Guest capacity', yaxis_title='Nightly price (USD)', legend_title='Host type')
price_scatter

### Nightly Price vs. Capacity
Bubble chart linking capacity, price, and host status to surface listings that punch above or below their weight.

**Why this matters**
- Reveals how prices scale with guest capacity across superhosts and regular hosts
- Highlights potential underpriced opportunities (high rating, lower-than-peers pricing)
- Shows where review quality aligns—or conflicts—with nightly rates

**Data preparation steps**
- Filtered to listings with non-missing price, accommodates, and review rating
- Sampled up to 2,000 listings for responsive rendering while keeping random-state reproducibility
- Encoded host type into `Superhost` vs `Regular host` labels for intuitive colouring
- Sized points by `review_scores_rating` to blend reputation into the visual story

**What the scatter shows**
Most Amsterdam listings cluster between 2–6 guests, with superhosts generally pricing slightly higher. Larger capacities show wider price dispersion, and oversized bubbles below the main trendline flag high-rated, potentially undervalued stays worth deeper investigation.

In [19]:
raw_reviews.columns.tolist()

['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments']

#### Review aggregation summary
Aggregate review counts, recency, and text snippets to capture guest feedback trends per listing.

In [20]:
# Aggregate review signals and prepare TF–IDF-ready corpus
reviews_sorted = raw_reviews.sort_values(['listing_id', 'date'])
# Summaries at the listing grain capture volume and recency dynamics
reviews_stats = reviews_sorted.groupby('listing_id').agg(
    review_count=('id', 'count'),
    reviews_last_90d=('date', lambda x: (x >= calendar['date'].max() - pd.Timedelta(days=90)).sum()),
    last_review_date=('date', 'max'),
    first_review_date=('date', 'min'),
)
# Translate timestamps into numeric recency metrics usable by models
reviews_stats['days_since_last_review'] = (
    calendar['date'].max() - reviews_stats['last_review_date']
).dt.days
reviews_stats['days_since_last_review'] = reviews_stats['days_since_last_review'].where(
    reviews_stats['last_review_date'].notna()
)
reviews_stats['review_recency_score'] = 1 / (1 + reviews_stats['days_since_last_review'].fillna(np.inf))
# Concatenate the latest 20 reviews per listing as sentiment input
reviews_recent_text = reviews_sorted.groupby('listing_id').apply(
    lambda df: ' '.join(df['comments'].astype(str).tail(20))
).rename('recent_review_text').to_frame()
reviews_features = reviews_stats.join(reviews_recent_text, how='left')
reviews_features.head(3)

,review_count,reviews_last_90d,last_review_date,first_review_date,days_since_last_review,review_recency_score,recent_review_text
listing_id,,,,,,,
27886,311,0,2025-09-07,2012-01-09,368,0.002710,It is a nice place to stay! I can recommend ⭐️...
28871,732,0,2025-09-07,2010-08-22,368,0.002710,Tivemos uma estadia maravilhosa! O apartamento...
29051,849,0,2025-09-08,2011-03-16,367,0.002717,A clean room in the middle of the best spot in...


#### TF-IDF feature creation
Vectorize the most recent review text into 50 TF-IDF sentiment features per listing.

In [21]:
# Generate TF–IDF embeddings on the aggregated review text
tfidf_vectorizer = TfidfVectorizer(max_features=50, stop_words='english')
# Replace missing review text with blanks prior to vectorisation
text_corpus = reviews_features['recent_review_text'].fillna('')
tfidf_matrix = tfidf_vectorizer.fit_transform(text_corpus)
# Prefix terms so downstream models know these came from TF–IDF
tfidf_feature_names = [f"tfidf_{token}" for token in tfidf_vectorizer.get_feature_names_out()]
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=reviews_features.index, columns=tfidf_feature_names)
# Join numeric review stats and sentiment vectors in one block
reviews_tfidf_features = pd.concat([reviews_features.drop(columns=['recent_review_text']), tfidf_df], axis=1)
reviews_tfidf_features.head(3)

,review_count,reviews_last_90d,last_review_date,first_review_date,days_since_last_review,review_recency_score,tfidf_amazing,tfidf_amsterdam,tfidf_apartment,tfidf_area,...,tfidf_restaurants,tfidf_sehr,tfidf_stay,tfidf_super,tfidf_time,tfidf_tram,tfidf_très,tfidf_und,tfidf_walk,tfidf_wonderful
listing_id,,,,,,,,,,,,,,,,,,,,,
27886,311,0,2025-09-07,2012-01-09,368,0.002710,0.068641,0.267251,0.0,0.066780,...,0.032932,0.086864,0.380115,0.000000,0.099444,0.000000,0.00000,0.041806,0.134393,0.246841
28871,732,0,2025-09-07,2010-08-22,368,0.002710,0.117862,0.166870,0.0,0.057334,...,0.000000,0.000000,0.285552,0.152071,0.113835,0.062317,0.00000,0.000000,0.000000,0.000000
29051,849,0,2025-09-08,2011-03-16,367,0.002717,0.000000,0.127052,0.0,0.174611,...,0.114811,0.000000,0.082824,0.051460,0.057781,0.000000,0.12625,0.072873,0.000000,0.000000


#### Calendar temporal features
Derive day-of-week, seasonality, and rolling availability windows to capture demand fluctuations.

In [22]:
# Calendar-level temporal features to capture demand cycles
calendar_features = calendar.sort_values(['listing_id', 'date']).copy()
# Breakdown of calendar dates into interpretable seasonal buckets
calendar_features['day_of_week'] = calendar_features['date'].dt.day_name()
calendar_features['day_of_week_idx'] = calendar_features['date'].dt.dayofweek
calendar_features['is_weekend'] = (calendar_features['day_of_week_idx'] >= 5).astype(int)
calendar_features['weekofyear'] = calendar_features['date'].dt.isocalendar().week.astype(int)
calendar_features['month'] = calendar_features['date'].dt.month
calendar_features['year'] = calendar_features['date'].dt.year
season_map = {12: 'winter', 1: 'winter', 2: 'winter', 3: 'spring', 4: 'spring', 5: 'spring', 6: 'summer', 7: 'summer', 8: 'summer', 9: 'fall', 10: 'fall', 11: 'fall'}
calendar_features['season'] = calendar_features['month'].map(season_map)
# Rolling features describe short-term supply dynamics for each listing
calendar_features['rolling_availability_7d'] = (
    calendar_features.groupby('listing_id')['available_flag']
    .rolling(window=7, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)
calendar_features['rolling_min_nights_30d'] = (
    calendar_features.groupby('listing_id')['minimum_nights']
    .rolling(window=30, min_periods=1)
    .median()
    .reset_index(level=0, drop=True)
)
calendar_features.head(3)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,available_flag,nights_available_pct,month,day_of_week,day_of_week_idx,is_weekend,weekofyear,year,season,rolling_availability_7d,rolling_min_nights_30d
3310185,27886,2025-09-11,f,NaN,NaN,3,30,0,0.046575,9,Thursday,3,0,37,2025,fall,0.0,3.0
3310186,27886,2025-09-12,f,NaN,NaN,3,30,0,0.046575,9,Friday,4,0,37,2025,fall,0.0,3.0
3310187,27886,2025-09-13,f,NaN,NaN,3,30,0,0.046575,9,Saturday,5,1,37,2025,fall,0.0,3.0


#### Categorical encoding helper
Create reusable logic to one-hot encode the most frequent values for interpretability.

In [23]:
# Helper to one-hot encode the most frequent categories for interpretability
import re
def encode_top_categories(df: pd.DataFrame, column: str, top_n: int = 8) -> pd.DataFrame:
    top_categories = df[column].value_counts(dropna=True).nlargest(top_n).index
    for category in top_categories:
        safe_name = re.sub(r'[^0-9a-zA-Z]+', '_', str(category)).lower().strip('_')
        df[f"{column}__{safe_name}"] = (df[column] == category).astype(int)
    df[f"{column}__other"] = (~df[column].isin(top_categories) & df[column].notna()).astype(int)
    return df
categorical_columns = [
    'property_type','room_type','neighbourhood_cleansed','season','day_of_week','host_response_time'
]

#### Host performance features
Score host responsiveness, occupancy, and revenue estimates to enrich booking predictors before assembling the ML table.

In [ ]:
# Additional listing signals to align with modeling needs
response_time_order = {
    'within an hour': 4,
    'within a few hours': 3,
    'within a day': 2,
    'a few days or more': 1,
}
# Human-readable response times become ordinal scores for models
listings['host_response_time_score'] = listings['host_response_time'].map(response_time_order)
listings['host_response_rate'] = listings['host_response_rate'].clip(0, 1)
listings['host_acceptance_rate'] = listings['host_acceptance_rate'].clip(0, 1)
# Occupancy and revenue estimates supplement availability target with economic context
if 'estimated_occupancy_l365d' in raw_listings.columns:
    listings['occupancy_estimate'] = to_float(raw_listings['estimated_occupancy_l365d'])
else:
    listings['occupancy_estimate'] = np.nan
if 'estimated_revenue_l365d' in raw_listings.columns:
    listings['revenue_estimate'] = parse_currency(raw_listings['estimated_revenue_l365d'])
else:
    listings['revenue_estimate'] = np.nan
# Remove verbose text columns after deriving numeric features
listings_select = listings.drop(columns=['amenities'])
listings_select.head(3)

#### Listing history features
Compute listing age and review presence flags to capture maturity and reputation ahead of the merge.

In [ ]:
# Temporal listing metadata derived from review history
listings_select['listing_age_days'] = (calendar['date'].max() - listings_select['first_review']).dt.days
listings_select['listing_age_days'] = listings_select['listing_age_days'].where(listings_select['first_review'].notna())
# Binary flag clarifies whether a listing has any review evidence at all
listings_select['has_reviews'] = listings_select['number_of_reviews'].gt(0).astype(int)
listings_select[['listing_id', 'listing_age_days', 'has_reviews']].head(3)

#### Dataset assembly
Join calendar, listing, and review features and apply top-category encodings to build the supervised table.

In [26]:
# Merge calendar, listing, and review blocks into a unified supervised table
ml_dataset = (
    calendar_features
    .merge(listings_select, on='listing_id', how='left', suffixes=('', '_listing'))
    .merge(reviews_tfidf_features, on='listing_id', how='left')
)
# Encode the most common categories while preserving interpretability
for column in categorical_columns:
    if column in ml_dataset.columns:
        ml_dataset = encode_top_categories(ml_dataset, column, top_n=8)
# Drop raw categorical columns that have been encoded or are redundant
columns_to_drop = [
    'available','property_type','room_type','neighbourhood_cleansed','season','day_of_week',
    'host_response_time','first_review','last_review'
 ]
ml_dataset = ml_dataset.drop(columns=[col for col in columns_to_drop if col in ml_dataset.columns])
# Quick preview to ensure merges and encodings produced the expected schema
feature_preview = ml_dataset.head(3)
ml_dataset.shape, feature_preview

((3825200, 148),
    listing_id       date  price  adjusted_price  minimum_nights  \
 0       27886 2025-09-11    NaN             NaN               3   
 1       27886 2025-09-12    NaN             NaN               3   
 2       27886 2025-09-13    NaN             NaN               3   
 
    maximum_nights  available_flag  nights_available_pct  month  \
 0              30               0              0.046575      9   
 1              30               0              0.046575      9   
 2              30               0              0.046575      9   
 
    day_of_week_idx  ...  day_of_week__sunday  day_of_week__monday  \
 0                3  ...                    0                    0   
 1                4  ...                    0                    0   
 2                5  ...                    0                    0   
 
    day_of_week__tuesday  day_of_week__wednesday  day_of_week__other  \
 0                     0                       0                   0   
 1           

#### Feature diagnostics
Profile class balance, feature counts, and missingness to validate the engineered matrix.

In [27]:
# Diagnostics for the engineered feature space
target_column = 'available_flag'
# Exclude the target and date columns when profiling feature health
feature_columns = [col for col in ml_dataset.columns if col not in {'date', target_column}]
missing_stats = ml_dataset[feature_columns].isna().mean().sort_values(ascending=False).head(10)
# Capture dataset breadth and class balance for documentation
dataset_profile = pd.DataFrame({
    'total_rows': [len(ml_dataset)],
    'feature_count': [len(feature_columns)],
    'target_positive_rate': [ml_dataset[target_column].mean()],
    'non_null_share_median': [1 - ml_dataset[feature_columns].isna().mean().median()],
})
dataset_profile, missing_stats

(   total_rows  feature_count  target_positive_rate  non_null_share_median
 0     3825200            146              0.257678               0.982968,
 adjusted_price                  1.000000
 neighbourhood_group_cleansed    1.000000
 price                           1.000000
 revenue_estimate                0.439504
 price_listing                   0.439504
 target_price                    0.439504
 beds                            0.436641
 host_response_time_score        0.362309
 host_response_rate              0.362309
 host_acceptance_rate            0.231870
 dtype: float64)

#### Prune sparse features
Drop columns with more than 90% missingness to keep the feature space efficient.

In [28]:
# Remove features that are mostly missing to keep the matrix compact
missing_ratio = ml_dataset.isna().mean()
high_missing_cols = missing_ratio[missing_ratio > 0.9].index.tolist()
# Eliminating near-empty columns reduces noise without hurting signal
ml_dataset = ml_dataset.drop(columns=high_missing_cols)
feature_columns = [col for col in ml_dataset.columns if col not in {'date', target_column}]
len(high_missing_cols), sorted(high_missing_cols)[:5]

(3, ['adjusted_price', 'neighbourhood_group_cleansed', 'price'])

#### Persist engineered datasets
Save the full feature matrix and a stratified 200k-row sample for downstream modeling.

In [ ]:
# Persist the full matrix and a stratified sample for quick iteration
output_parquet = DATA_DIR / 'airbnb_ml_ready_dataset.parquet'
output_sample = DATA_DIR / 'airbnb_ml_ready_sample.parquet'
# Full dataset supports high-capacity models; sample enables rapid prototyping
ml_dataset.to_parquet(output_parquet, index=False)
sample_size = 200_000
sampled_dataset = (
    ml_dataset.groupby(target_column, group_keys=False)
    .apply(lambda g: g.sample(min(len(g), sample_size // 2), random_state=42))
    .reset_index(drop=True)
)
sampled_dataset.to_parquet(output_sample, index=False)
output_parquet, output_sample, sampled_dataset.shape

(WindowsPath('c:/Users/frunc/Desktop/ttt/airbnb_ml_ready_dataset.parquet'),
 WindowsPath('c:/Users/frunc/Desktop/ttt/airbnb_ml_ready_sample.parquet'),
 (200000, 145))

#### Feature catalog snapshot
Document column names, dtypes, and missingness to aid future modelers.

### Reasoning Journey
1. **Understand supply & demand** – monthly and weekday availability plots validate the temporal drivers that the target must capture before modeling.
2. **Quantify price levers** – price histograms and the capacity vs. price scatter highlight how size, reputation, and host quality influence nightly rates.
3. **Engineer predictive signal** – cleaned numerical features, categorical encodings, and TF–IDF vectors translate raw text and metadata into machine-friendly inputs.
4. **Guard data quality** – diagnostics, sparse-column pruning, and feature catalogs ensure the final matrix is compact, well-documented, and ready for modeling.
5. **Enable experimentation** – exporting a stratified sample accelerates iteration while the full parquet preserves maximum signal for production training.

In [ ]:
# Feature catalog to help downstream modeling
metadata = pd.DataFrame({
    'feature': feature_columns,
    'dtype': ml_dataset[feature_columns].dtypes.astype(str),
    'missing_pct': ml_dataset[feature_columns].isna().mean(),
})
# Sorting by missingness highlights columns that may need imputation strategies
metadata.sort_values('missing_pct').head(20)

,feature,dtype,missing_pct
listing_id,listing_id,int64,0.0
neighbourhood_cleansed__de_pijp_rivierenbuurt,neighbourhood_cleansed__de_pijp_rivierenbuurt,int64,0.0
neighbourhood_cleansed__centrum_west,neighbourhood_cleansed__centrum_west,int64,0.0
neighbourhood_cleansed__de_baarsjes_oud_west,neighbourhood_cleansed__de_baarsjes_oud_west,int64,0.0
room_type__other,room_type__other,int64,0.0
room_type__shared_room,room_type__shared_room,int64,0.0
room_type__hotel_room,room_type__hotel_room,int64,0.0
room_type__private_room,room_type__private_room,int64,0.0
room_type__entire_home_apt,room_type__entire_home_apt,int64,0.0
property_type__other,property_type__other,int64,0.0
